# Cloud Logging with Weights & Biases

This tutorial describes how to log the information generated during model development to the cloud.
While many 3rd-party cloud solutions are available, this tutorial uses [Weights & Biases](https://wandb.ai):

> Weights & Biases makes it easy to track your experiments, manage & version your data, and collaborate with your team so you can focus on building the best models.

This has the following benefits:
- All model development files (trained model files, evaluation results, etc.) are saved to the cloud
- Graphical interface for comparing different training runs
- Sort training runs by custom configuration values to see which has the most benefit
- View model training status from anywhere 
- Email alerts for when model training completes
- Automatically generate model report (interactive webpage or PDF)
- Monitor GPU and CPU usage during model training

![](https://github.com/SiliconLabs/mltk/raw/master/docs/img/wandb_overview.gif)

## Disclaimer

[Weights & Biases](https://wandb.ai) is free for personal use but requires a paid subscription for commercial use.

See the [Pricing Guide](https://wandb.ai/site/pricing) for more information.

## Quick Links

- [Weight & Biases Documentation](https://docs.wandb.ai)
- [WeightsAndBiasesMixin API Reference](https://siliconlabs.github.io/mltk/docs/python_api/mltk_model/weights_and_biases_mixin.html)

## Contents

This tutorial is divided into the following sections:

1. [Basic Usage](#basic-usage)
2. [Logging information](#logging-information)
3. [Training and logging in the cloud](#training-and-logging-in-the-cloud)
4. [Other Features](#other-features)

## Basic Usage

The basic flow for logging information to the cloud is as follows:

1. Create a Weights & Biases [account](https://app.wandb.ai/login?signup=true)
2. Install the MLTK, see the [Installation Guide](https://siliconlabs.github.io/mltk/docs/installation.html#standard-python-package):  
   ```shell
   pip install silabs-mltk
   ```

3. Install the [Weights & Biases](https://docs.wandb.ai/quickstart) python package:   
   ```shell
   pip install wandb
   ```
4. Log into your wandb account using the command:  
   ```shell
   wandb login
   ```
5. Define your MLTK [model specification]() and extend the [WeightsAndBiasesMixin](https://siliconlabs.github.io/mltk/docs/python_api/mltk_model/weights_and_biases_mixin.html), e.g.:  

   ```python
   import mltk.core as mltk_core

   class MyModel(
      mltk_core.MltkModel,
      mltk_core.TrainMixin,
      mltk_core.DatasetMixin,
      mltk_core.EvaluateClassifierMixin,
      mltk_core.WeightsAndBiasesMixin
   ):
      pass
   my_model = MyModel()
   ```
6. [Train](https://siliconlabs.github.io/mltk/docs/guides/model_training.html) your model with the command:  
   ```shell
   mltk train <model name> --post
   ```
   Where `--post` tells the MLTK to enable the [WeightsAndBiasesMixin](https://siliconlabs.github.io/mltk/docs/python_api/mltk_model/weights_and_biases_mixin.html)
7. Optionally [profile](https://siliconlabs.github.io/mltk/docs/guides/model_profiler.html) your model with the command:    
   ```shell
   mltk profile <model name> --accelerator mvp --device --post
   ```
   Where `--post` tells the MLTK to enable the [WeightsAndBiasesMixin](https://siliconlabs.github.io/mltk/docs/python_api/mltk_model/weights_and_biases_mixin.html)


That's it! This will train (and optionally profile) the model and all relevant information will automatically be logged to the [Weights & Biases](https://wandb.ai) cloud.

## Logging Information

The [WeightsAndBiasesMixin](https://siliconlabs.github.io/mltk/docs/python_api/mltk_model/weights_and_biases_mixin.html) manages all of the details of initializing the wandb session and automatically logging the relavant information to the cloud.

### Default Data

By default, the following information is logged to the cloud:

- [Model specification](https://siliconlabs.github.io/mltk/docs/guides/model_specification.html) - The Python script that defines your model
- [Model archive](https://siliconlabs.github.io/mltk/docs/guides/model_archive.html) - ZIP file containing the trained model files 
- [Model summary](https://siliconlabs.github.io/mltk/docs/guides/model_summary.html) - Basic informaation about the model
- Dataset summary - Basic information about the dataset used for training and evaluation
- [Training history](https://docs.wandb.ai/ref/python/integrations/keras/wandbcallback) - Learn rate, accuracy, loss, etc.
- [Evaluation results](https://siliconlabs.github.io/mltk/docs/guides/model_evaluation.html) - Precision, recall, ROC, etc.
- [Profiling results](https://siliconlabs.github.io/mltk/docs/guides/model_profiler.html) - Inference/s, RAM and flash usage, etc.
- System usage - GPU, CPU, disk, memory utilization, etc.

### Custom Data

Any custom data can also be stored in the cloud using:
- [WeightsAndBiasesMixin.wandb_log()](https://siliconlabs.github.io/mltk/docs/python_api/mltk_model/weights_and_biases_mixin.html#mltk.core.WeightsAndBiasesMixin.wandb_log) - post custom logging information
- [WeightsAndBiasesMixin.wandb_save()](https://siliconlabs.github.io/mltk/docs/python_api/mltk_model/weights_and_biases_mixin.html#mltk.core.WeightsAndBiasesMixin.wandb_save) - upload a file

Internally, these APIs call the wandb APIs:
- [wandb.log()](https://docs.wandb.ai/ref/python/log)
- [wandb.save()](https://docs.wandb.ai/ref/python/save)


## Training and logging in the cloud

The MLTK supports training via [SSH](https://siliconlabs.github.io/mltk/docs/guides/model_training_via_ssh.html).
This feature allows for model training in the cloud which can greatly reduce model training time.  
See the [Cloud Training via vast.ai](https://siliconlabs.github.io/mltk/mltk/tutorials/cloud_training_with_vast_ai.html) tutorial for more details.

The following is needed to enable logging to [Weight & Biases](https://wandb.ai) while training in the cloud:

1. Obtain your wandb [API Key](https://docs.wandb.ai/guides/integrations/dagster#how-to-get-an-api-key):  
   a. [Log in to W&B](https://wandb.ai/login)  
   b. Collect your API key by navigating to the [authorization page](https://wandb.ai/authorize)
2. Create/modify the [~/.mltk/user_settings.yaml](https://siliconlabs.github.io/mltk/docs/other/settings_file.html) file and add:  
   ```yaml
   ssh:
      startup_cmds:
      - pip install wandb plotly
      - wandb login <API KEY> 
   ```
   where `<API KEY>` is your API key from step 1

That's it! When you model trains on the cloud machine, it will also log to the Weights & Biases cloud.  
This is especially useful when used in conjunction with the [Alerts](https://docs.wandb.ai/guides/track/alert) feature of W&B so you can be notified when training completes.

## Other Features

The Weights & Biases [documentation](https://docs.wandb.ai/guides) describes many useful features.

Some of the more useful features include:

- [Collaborative Reports](https://docs.wandb.ai/guides/reports) - Reports let you organize, embed and automate visualizations, describe your findings, share updates with collaborators, and more.
- [Alerts](https://docs.wandb.ai/guides/track/alert) - Send alerts, triggered from your Python code, to your Slack or email
- [Storing Artifacts](https://docs.wandb.ai/guides/artifacts) -  track datasets, models, dependencies, and results through each step of your machine learning pipeline. Artifacts make it easy to get a complete and auditable history of changes to your files.